# Callback: Custom Resource

Consider again the Vehicle Routing Problem with Time Windows but let's do the `time` resource ourselves.

Hook into the dynamic programming algorithm and do initialization, extension and dominance manually.

In [ ]:
# Vehicle Routing Problem with Time Windows

from flowty import Model, xsum, CallbackModel, Where
from flowty.datasets import vrp_rep

bunch = vrp_rep.fetch_vrp_rep("solomon-1987-r1", instance="R101_025")
name, n, es, c, d, Q, t, a, b, x, y = bunch["instance"]

In [ ]:
m = Model()

# Make sure to invoke callback in the dynamic progrmaming algorithm
m.setParam("CallbackDP", "On")

# one graph, it is identical for all vehicles
g = m.addGraph(obj=c, edges=es, source=0, sink=n - 1, L=1, U=n - 2, type="B")

# adds resources variables to the graph.
# demand and capacity
m.addResourceDisposable(
    graph=g, consumptionType="V", weight=d, boundsType="V", lb=0, ub=Q, name="d"
)

# A custom resource to handle time
m.addResourceCustom(graph=g, name="time")

# The callback for handling the time resource
def callback(cb: CallbackModel, where: Where):
    # initialization
    if where == Where.DPInit:
        cb.setResource("time", 0)

    # extension
    if where == Where.DPExtend:
        e = cb.edge
        j = es[e][1]
        value = cb.getResource("time")
        value = max(a[j], value + t[e])

        if value > b[j]:
            cb.skip()
        else:
            cb.setResource("time", value)

    # dominance
    if where == Where.DPDominate:
        value = cb.getResource("time")
        other = cb.getResourceOther("time")

        # label is not dominated
        if other < value:
            cb.keep()


m.setCallback(callback)

# set partition constriants
for i in range(n)[1:-1]:
    m += xsum(x * 1 for x in g.vars if i == x.source) == 1

# packing set
for i in range(n)[1:-1]:
    m.addPackingSet([x for x in g.vars if i == x.source])

status = m.optimize()

print(f"ObjectiveValue {m.objectiveValue}")

In [ ]:
# Display
import networkx
import matplotlib
import matplotlib.pyplot as plt
from flowty import OptimizationStatus
import math

if (
    status == OptimizationStatus.Optimal
    or status == OptimizationStatus.Feasible
):
    edges = [x.edge for x in g.vars if not math.isclose(x.x, 0, abs_tol=0.001)]
    
    plt.figure(figsize=(20,20))

    g = networkx.DiGraph()
    g.add_nodes_from([i for i in range(n)])
    g.add_edges_from(edges)
    pos = {i: (x[i], y[i]) for i in range(n)}

    networkx.draw_networkx_nodes(g, pos, nodelist=g.nodes)
    labels = {i: i for i in g.nodes}
    networkx.draw_networkx_labels(g, pos, labels=labels)

    networkx.draw_networkx_edges(g, pos, nodelist=g.edges)

    plt.show()